# Notebook 04: Retrocausal Models and Bell Tests

**References**:
- Price (2008), "Toy Models for Retrocausality"
- Wharton & Argaman (2020), Rev. Mod. Phys. 92, 021002
- Bell (1964), Physics 1, 195-200

## The Bell Inequality Trilemma

Bell's theorem (1964) shows that **no local hidden variable model** can reproduce all quantum predictions. You must give up at least one of:

1. **Locality**: outcomes depend only on local variables
2. **Measurement independence**: hidden variables are independent of future settings
3. **Realism**: outcomes are predetermined by hidden variables

Standard QM gives up **locality** (nonlocal wavefunction collapse). Retrocausal models take a different path: they give up **measurement independence** while keeping locality.

The hidden variable $\lambda$ at the source depends on **both future measurement settings** — but each party's outcome still depends only on their local variables.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from src.retrocausal.zigzag_model import ZigZagModel, ClassicalLocalModel
from src.retrocausal.boundary_value import BoundaryValueModel
from src.retrocausal.bell_test import BellTestComparator
from src.retrocausal.no_signaling_audit import NoSignalingAudit
from src.analysis.bell_inequality import qm_singlet_correlation

## 1. The CHSH Inequality

The CHSH (Clauser-Horne-Shimony-Holt) inequality:

$$S = E(a,b) - E(a,b') + E(a',b) + E(a',b')$$

- **Classical bound**: $|S| \leq 2$
- **Quantum bound (Tsirelson)**: $|S| \leq 2\sqrt{2} \approx 2.828$
- **QM prediction for singlet with optimal settings**: $S = -2\sqrt{2}$

Let's compare three models:

In [ ]:
# Compare CHSH values across models
comparator = BellTestComparator(n_trials=50000)
chsh = comparator.compute_chsh_values()

print("CHSH Bell Test Results")
print("=" * 60)
print(f"{'Model':<25} {'S value':>10} {'|S|':>8} {'Violates?':>10}")
print("-" * 60)
for name, S in chsh.items():
    violates = 'YES' if abs(S) > 2.0 else 'no'
    print(f"{name:<25} {S:>+10.4f} {abs(S):>8.4f} {violates:>10}")
print("-" * 60)
print(f"{'Classical bound':<25} {'':>10} {'2.000':>8}")
print(f"{'Tsirelson bound':<25} {'':>10} {'2.828':>8}")

## 2. Correlation Curves: E(0, theta)

The singlet state correlation is $E(a,b) = -\cos(a-b)$. Let's see how each model compares.

In [ ]:
# Sweep E(0, theta) for all models
n_angles = 36
df = comparator.run_chsh_sweep(n_angles=n_angles)

fig, ax = plt.subplots(figsize=(10, 6))

styles = {
    'QM (analytical)': ('black', '-', 3),
    'ZigZag Retrocausal': ('blue', 'o', 1),
    'Boundary-Value': ('green', 's', 1),
    'Classical Local': ('red', '^', 1),
}

for model_name, (color, marker, lw) in styles.items():
    data = df[df['model'] == model_name]
    if marker in ['-']:
        ax.plot(np.degrees(data['angle']), data['correlation'], 
                color=color, linewidth=lw, label=model_name)
    else:
        ax.plot(np.degrees(data['angle']), data['correlation'],
                color=color, marker=marker, markersize=4, linewidth=lw,
                alpha=0.7, label=model_name)

# Add the classical linear prediction for reference
theta_deg = np.linspace(0, 360, 200)
theta_rad = np.radians(theta_deg)
E_qm = -np.cos(theta_rad)
ax.plot(theta_deg, E_qm, 'k-', linewidth=2, alpha=0.3, label='QM: -cos(theta)')

ax.set_xlabel('Angle theta (degrees)', fontsize=12)
ax.set_ylabel('Correlation E(0, theta)', fontsize=12)
ax.set_title('Bell Correlation Curves: Retrocausal vs Classical vs QM', fontsize=13)
ax.legend(fontsize=10, loc='upper right')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='gray', linewidth=0.5)
plt.tight_layout()
plt.show()

## 3. The ZigZag Mechanism Explained

How does the ZigZag model violate Bell's inequality while remaining local?

```
Time
  |
  |  Alice measures a     Bob measures b
  |       |                    |
  |       v                    v
  |  A = f(lambda_A, a)   B = g(lambda, b)     ← LOCAL outcomes
  |       |                    |
  |       +----> SOURCE <------+
  |           lambda ~ p(lambda | a, b)          ← RETROCAUSAL
  |                                                 (future-input dependent)
  v
```

The key: $p(\lambda | a, b)$ depends on **both** future settings. But each outcome is still local!

In [ ]:
# Demonstrate the retrocausal mechanism
zigzag = ZigZagModel(n_trials=100000)

# Show that correlation matches QM
test_angles = [(0, np.pi/4), (0, np.pi/2), (0, np.pi), (np.pi/3, np.pi/6)]

print("ZigZag Model: Correlation vs QM Prediction")
print("=" * 55)
print(f"{'(a, b)':>20} {'E_zigzag':>12} {'E_QM':>12} {'Error':>10}")
print("-" * 55)

for a, b in test_angles:
    E_model = zigzag.correlation(a, b)
    E_qm = qm_singlet_correlation(a, b)
    err = abs(E_model - E_qm)
    print(f"({a:.2f}, {b:.2f}){' ':>8} {E_model:>+12.4f} {E_qm:>+12.4f} {err:>10.4f}")

## 4. No-Signaling Audit

The **critical** check: even though the retrocausal model uses future-input dependent hidden variables, it must still satisfy no-signaling. Alice's marginal $P(A|a)$ must be independent of Bob's setting $b$.

In [ ]:
# Run no-signaling audit on all models
audit = NoSignalingAudit(n_trials=50000)
results = audit.audit_all_models()

print("No-Signaling Audit Results")
print("=" * 65)

for model_name, result in results.items():
    status = 'PASSED' if result['no_signaling'] else 'FAILED'
    print(f"\n  {model_name}: {status}")
    print(f"    Max marginal deviation from 0.5: {result['max_deviation']:.4f}")
    for key, val in result['alice_marginals'].items():
        print(f"      P(A=+1 | a=0, {key}) = {val:.4f}")

## 5. Boundary-Value Model

The Wharton-Argaman boundary-value approach treats the experiment as a **constraint satisfaction problem**:

- **Initial boundary**: entangled state at the source
- **Final boundary**: measurement outcomes at the detectors
- **Physics in between**: determined by the action principle, constrained by BOTH boundaries

This is analogous to Lagrangian mechanics vs Newtonian mechanics.

In [ ]:
# Demonstrate the boundary-value action landscape
bv = BoundaryValueModel(n_paths=50000)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

settings = [(0, np.pi/4), (0, np.pi/2), (0, np.pi)]

for idx, (a, b) in enumerate(settings):
    lam_vals, action = bv.action_landscape(a, b, n_lambda=300)
    axes[idx].plot(np.degrees(lam_vals), action, 'b-', linewidth=1.5)
    axes[idx].set_xlabel('lambda (degrees)', fontsize=11)
    axes[idx].set_ylabel('Action S(lambda)', fontsize=11)
    axes[idx].set_title(f'a={a:.2f}, b={b:.2f}\nE = {bv.correlation(a, b):.3f}', fontsize=11)
    axes[idx].grid(True, alpha=0.3)
    # Mark minima
    min_idx = np.argmin(action[10:-10]) + 10  # avoid edges
    axes[idx].plot(np.degrees(lam_vals[min_idx]), action[min_idx], 'ro', markersize=8)

plt.suptitle('Boundary-Value Model: Action Landscape S(lambda | a, b)',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print("The action minima shift as boundary conditions (a, b) change.")
print("This is the retrocausal mechanism: future measurements constrain past states.")

## 6. Classical Model Comparison

A classical local hidden variable model (no retrocausality) **cannot** violate Bell's inequality. The classical model uses $\lambda$ uniformly distributed, independent of future settings.

In [ ]:
classical = ClassicalLocalModel(n_trials=100000)

# Classical correlation curve
angles = np.linspace(0, 2*np.pi, 50)
E_classical = [classical.correlation(0, th) for th in angles]
E_qm = -np.cos(angles)
E_linear = np.where(angles <= np.pi, 
                     -1 + 2*angles/np.pi,
                     3 - 2*angles/np.pi)

fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(np.degrees(angles), E_qm, 'k-', linewidth=2, label='QM: -cos(theta)')
ax.plot(np.degrees(angles), E_classical, 'r.', markersize=4, label='Classical model')
ax.plot(np.degrees(angles), E_linear, 'r--', alpha=0.5, label='Classical: -1 + 2|theta|/pi')
ax.set_xlabel('Angle theta (degrees)', fontsize=12)
ax.set_ylabel('Correlation E(0, theta)', fontsize=12)
ax.set_title('Classical vs QM: The Gap That Bell Identified', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

result = classical.run_bell_test()
print(f"Classical CHSH: S = {result.chsh_value:.4f} (bound: |S| <= 2)")
print(f"Violates Bell: {abs(result.chsh_value) > 2.0}")

## 7. Full Comparison Report

In [ ]:
report = comparator.generate_comparison_report()
print(report.summary)

## Key Takeaways

1. **Retrocausal models can violate Bell's inequality while being local** — the trick is allowing hidden variables to depend on future measurement settings.

2. **Classical local models (no future-input dependence) cannot violate Bell's inequality** — this is Bell's theorem.

3. **All models respect no-signaling** — Alice's marginal distribution is always independent of Bob's setting. No information flows backward.

4. **The boundary between "retrocausal correlations" and "time travel" is precise**: correlations in the joint distribution can depend on both settings, but marginals cannot.

5. **The retrocausal interpretation is a legitimate alternative** to nonlocal collapse — it trades "spooky action at a distance" for "future-input dependent hidden variables."

---

**Next**: [05_advanced_experiments.ipynb](05_advanced_experiments.ipynb) — GHZ states, decoherence, phase transitions, and quantum speedup.